In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import torch
import numpy as np
from sklearn.svm import SVC, NuSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report

In [2]:
t_train = torch.load('../data/embeds/train_polyhope_english_bbu_binary.pt')
t_val = torch.load('../data/embeds/val_polyhope_english_bbu_binary.pt')
t_test = torch.load('../data/embeds/polyhope_test_english_bbu_binary.pt')

s_train = torch.load('../data/embeds/polyhope_train_english_sentiment.pt')
s_val = torch.load('../data/embeds/polyhope_val_english_sentiment.pt')
s_test = torch.load('../data/embeds/polyhope_test_english_sentiment.pt')

In [3]:
polyhope_multi_labels = {'Not Hope':1, 'Generalized Hope':2, 'Realistic Hope':3, 'Unrealistic Hope':4}
polyhope_inv_multi_labels = {v: k for k, v in polyhope_multi_labels.items()}
polyhope_binary_labels = {'Hope':1, 'Not Hope':0}
polyhope_inv_binary_labels = {v: k for k, v in polyhope_binary_labels.items()}

In [4]:
df_train = pd.read_csv('../data/train_polyhope_english.csv')
train_bin = df_train['binary'].tolist()
train_multi = df_train['multiclass'].tolist()

df_test = pd.read_csv('../data/polyhope_test_english.csv')
test_texts = df_test['text'].tolist()
test_ids = df_test['id'].tolist()

df_val = pd.read_csv('../data/val_polyhope_english.csv')
val_texts = df_val['text']
val_bin = df_val['binary'].tolist()
val_multi = df_val['multiclass'].tolist()

In [5]:
t_train = np.array([x[0].numpy() for x in t_train])
t_val = np.array([x[0].numpy() for x in t_val])
t_test = np.array([x[0].numpy() for x in t_test])
s_train = np.array([x[0].numpy() for x in s_train])
s_val = np.array([x[0].numpy() for x in s_val])
s_test = np.array([x[0].numpy() for x in s_test])

In [27]:
# X = np.concatenate((t_train, s_train), axis=1)
X = (t_train + s_train)/2
Y = [polyhope_binary_labels[i] for i in train_bin]

clf = SVC()

clf.fit(t_train, Y)

SVC()

In [28]:
# X_test = np.concatenate((t_val, s_val), axis=1)
X_test = (t_val + s_val)/2
Y_test = [polyhope_binary_labels[i] for i in val_bin]
clf.score(t_val, Y_test)

0.8575581395348837

In [13]:
pred = []
# val_bin_c = [polyhope_multi_labels[i] for i in val_multi]
for i, text in enumerate(tqdm(test_texts)):
    pred.append(clf.predict([np.concatenate((t_test[i], s_test[i]), axis=None)]))

outs = [polyhope_inv_binary_labels[i[0]] for i in pred]

df_out = pd.DataFrame(zip(test_ids, outs), columns=['id', 'category'])
df_out.to_csv('../result_english_binary_with_sentiment.csv', index=False)
# print(accuracy_score(val_bin_c, pred))
# print(classification_report(val_bin_c, pred))

100%|██████████████████████████████████████████████████████████| 1032/1032 [00:00<00:00, 1972.63it/s]


In [15]:
from collections import Counter
a = Counter(outs)

In [25]:
a = np.array([1, 2])
b = np.array([3, 4])
a+b/2

array([2.5, 4. ])

In [26]:
(a+b)/2

array([2., 3.])